# Проект

### оставь надежду, всяк сюда входящий - возьми лучше корпус

ячейка для импортов

In [ ]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from nltk import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

import pandas as pd

import numpy as np

ua = UserAgent(verify_ssl=False)
session = requests.session()

Ячейка для обкачивания ссылок на новости с сайта гордой газеты Московский комсомолец, а точнее, ее раздела "проишествия"

In [ ]:
news = []
page_number = 0
for i in tqdm(range(4)):
    page_number += 1
    url = f'https://www.mk.ru/incident/{page_number}/'
    req = session.get(url, headers={'User-Agent': ua.random})
    print(req)
    req.encoding = 'utf-8'
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    each_news = soup.find_all('div', {'class': 'listing-item__content'})
    if each_news:
        print(len(each_news))
    for one_piece in each_news:
        link = one_piece.find('a', {'class': 'listing-preview__content'}).attrs['href']
        news.append(link)

 25%|█████████████████████                                                               | 1/4 [00:00<00:01,  2.48it/s]

<Response [200]>
30


 50%|██████████████████████████████████████████                                          | 2/4 [00:00<00:00,  2.53it/s]

<Response [200]>
30


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:01<00:00,  2.29it/s]

<Response [200]>
30


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.36it/s]

<Response [200]>
30


In [ ]:
print(len(news))

120


Глянем, наберется ли в каждой из этой сотни новостей нужное количество слов, заодно посмотрим, что там происходит

In [ ]:
link = news[10]
req = session.get(link, headers={'User-Agent': ua.random})
req.encoding = 'utf-8'
req = req.text
req = BeautifulSoup(req, 'html.parser')
new_text = req.find('div', {'class': "article__body"}) #.text.lower()
#print(new_text)
new_text = new_text.find_all('p') #.text.lower()
print(new_text)

[<p>Мать пострадавших по профессии повар, отец - дворник, оба приехали из Киргизии. «Двушку» на Зеленом проспекте они сняли год назад. Когда началась пандемия, мужчина потерял большую часть заработка, поэтому платить аренду стало тяжело. На семейном совете было решено сдать одну комнату какому-нибудь земляку.  </p>, <p>В начале сентября нашелся приезжий из Ошской области. Мужчина заявил, что он аниматор и что к нему скоро приедет жена с ребенком. Позже выяснилось, что он придумал эту историю - на самом деле он одинок.</p>, <p>Об ужасной ситуации 29-летняя мать девочек 6 и 7 лет от роду узнала совершенно случайно - 19 октября во время заучивания с детьми стихов. Старшая девочка, увидев соседа по квартире, вышедшего из комнаты в туалет, внезапно замкнулась и стала в волнении грызть ногти, заявив, что боится этого дядю. Женщина поинтересовалась происходящим, но девочка не ответила. </p>, <p>А вторая дочка, более открытая по характеру, как на духу выложила неприятные подробности. Она расск

Что и ожидалось от раздела Проишествия. Но тут точно есть сто слов!

Пойдем используем собранный список ссылок и обкачаем сайт нормально

In [ ]:
def get_full_news(link_list):
    texts = []
    titles = []
    for link in tqdm(link_list):
        news = []
        req = session.get(link, headers={'User-Agent': ua.random})
        req.encoding = 'utf-8'
        req = req.text
        req = BeautifulSoup(req, 'html.parser')
        title = req.find('h1', {'class': "article__title"})
        titles.append(title)
        new_text = req.find('div', {'class': "article__body"})
        new_text = new_text.find_all('p')
        for p in new_text:
            p = p.text
            news.append(p)
        texts.append(news)
    return titles, texts

In [ ]:
titles, texts = get_full_news(news)
if len(texts)==len(titles):
    print('all clear!')

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:34<00:00,  3.51it/s]

all clear!


Ячейка для предобработки. Не удивляйтесь, почему я тут токенизирую, а потом передумываю - так надо, такой датафрейм специфический

In [ ]:
def texts_process(texts):
    new_texts = []
    for text in texts:
        new_text = ''
        for paragraph in text:
            if not paragraph.startswith('\n') and not paragraph.startswith('Читайте также:'):
                new_text = new_text + paragraph +' '
                
        new_text = word_tokenize(new_text)
        new_text = ' '.join(new_text)
        new_texts.append(new_text)
    return new_texts

К заголовкам у нас не будет особых требований, мы по ним просто ориентируемся, так что просто сделаем служебную скучную функцию с непонятными переменными, чтобы преобразовывать их из типа Тег в тип Строка

In [ ]:
def title_process(texts):
    new_texts = []
    for text in texts:
        text = text.text
        new_texts.append(text)
    return new_texts

Обработаем данные и соберем в наш первый датафрейм

In [ ]:
clear_texts = texts_process(texts)
clear_titles = title_process(titles)
df = pd.DataFrame(list(zip(clear_titles, clear_texts)), 
               columns =['titles', 'full_texts']) 
df

,titles,full_texts
0,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста..."
1,"Девочка сбежала от матери, делавшей из нее рабыню","Наверное , у каждого были прадедушки и прабабу..."
2,Медбрат украл аппарат УЗИ у больных коронавирусом,"Как стало известно « МК » , пропажу аппарата У..."
3,Составлен рейтинг мошенничеств с коронавирусом,Лже-медики . Облапошивание начинается со звонк...
4,Приговор обрадовал депутата-коммуниста Шеремет...,Ранее в ходе прений прокурор потребовал для Ше...
...,...,...
115,Подросток умер во время вечеринки на юго-запад...,"Как стало известно « МК » , инцидент произошел..."
116,Российский полицейский убил любовницу-трансген...,"По информации издания , поначалу мужчина пытал..."
117,Студент МГИМО покончил с собой из-за сессии,"По информации агентства , его тело обнаружили ..."
118,Рабочего-расчленителя задержали в Петербурге,"По версии следственных органов , 5 сентября ме..."


Это здорово! Сохраним, на всякий случай

In [ ]:
df.to_csv('textbase_news_articles.csv')

А вот сейчас веселье и пойдет. Эта функция служит для того, чтобы разделить полные тексты по предложениям с созданием новой строки датафрейма, у которой будет идентичный заголовок

In [ ]:
lst_col = 'full_texts' 

x = df.assign(**{lst_col:df[lst_col].str.split('.')})

my_dataframe = pd.DataFrame({col:np.repeat(x[col].values, x[lst_col].str.len()) for col in x.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

In [ ]:
my_dataframe

,titles,full_texts
0,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста..."
1,Раскрыта тайна препарата в организме утонувшей...,"Девушка сразу направилась в бассейн , где око..."
2,Раскрыта тайна препарата в организме утонувшей...,В бессознательном состоянии Софию вытащили из...
3,Раскрыта тайна препарата в организме утонувшей...,Смерть дочери знаменитого актера получила шир...
4,Раскрыта тайна препарата в организме утонувшей...,"Например , у родственников погибшей возникли ..."
...,...,...
2461,Футболисты забили до смерти одноклубника из-за...,"Игрока доставили в больницу , где он умер спу..."
2462,Футболисты забили до смерти одноклубника из-за...,Полиция начало расследование
2463,Футболисты забили до смерти одноклубника из-за...,"Отмечается , что матч между командами носил т..."
2464,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м..."


Она работает! То же самое, только делим предложения на слова с созданием новых строк - но с теми же заголовками и предложениями

In [ ]:
my_dataframe['words'] = my_dataframe['full_texts']
lst_col = 'words' 

x = my_dataframe.assign(**{lst_col:my_dataframe[lst_col].str.split(' ')})

my_final_dataframe = pd.DataFrame({col:np.repeat(x[col].values, x[lst_col].str.len()) for col in x.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

In [ ]:
my_final_dataframe

,titles,full_texts,words
0,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",Напомним
1,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",","
2,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",23-го
3,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",сентября
4,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",вечером
...,...,...,...
42672,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",убила
42673,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",10
42674,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",игроков
42675,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",


Чудно. Стаковерфлоу творит чудеса. Скормим эти чудеса пайморфи и создадим для важной нам выдачи еще три столбца

In [ ]:
my_final_dataframe['lemma'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].normal_form, axis = 1)
my_final_dataframe['form'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].tag, axis = 1)
my_final_dataframe['POS'] = my_final_dataframe.apply(lambda row: morph.parse(row.words)[0].tag.POS, axis = 1)

In [ ]:
my_final_dataframe

,titles,full_texts,words,lemma,form,POS
0,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",Напомним,напомнить,"VERB,perf,tran plur,1per,futr,indc",VERB
1,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",",",",",PNCT,None
2,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",23-го,23-го,"NOUN,inan,neut,Fixd sing,nomn",NOUN
3,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",сентября,сентябрь,"NOUN,inan,masc sing,gent",NOUN
4,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",вечером,вечером,ADVB,ADVB
...,...,...,...,...,...,...
42672,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",убила,убить,"VERB,perf,tran femn,sing,past,indc",VERB
42673,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",10,10,"NUMB,intg",None
42674,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",игроков,игрок,"NOUN,anim,masc plur,gent",NOUN
42675,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",,,UNKN,None


Казалось бы, все хорошо, но как же - у нас прорехи в датафрейме, помешают рядом стоящие слова замечать. Исправим.

In [ ]:
my_final_dataframe.drop(my_final_dataframe[my_final_dataframe.words == ''].index, inplace=True)

In [ ]:
my_final_dataframe

,titles,full_texts,words,lemma,form,POS
0,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",Напомним,напомнить,"VERB,perf,tran plur,1per,futr,indc",VERB
1,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",",",",",PNCT,None
2,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",23-го,23-го,"NOUN,inan,neut,Fixd sing,nomn",NOUN
3,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",сентября,сентябрь,"NOUN,inan,masc sing,gent",NOUN
4,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",вечером,вечером,ADVB,ADVB
...,...,...,...,...,...,...
42670,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",Уганде,уганда,"NOUN,inan,femn,Sgtm,Geox sing,datv",NOUN
42671,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",молния,молния,"NOUN,inan,femn sing,nomn",NOUN
42672,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",убила,убить,"VERB,perf,tran femn,sing,past,indc",VERB
42673,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",10,10,"NUMB,intg",None


И все бы ладно, да знаки препинания!

In [ ]:
punkt = [',', ';', ':', '"']
my_final_dataframe.drop(my_final_dataframe[my_final_dataframe.words.isin (punkt)].index, inplace=True)
my_final_dataframe

,titles,full_texts,words,lemma,form,POS
0,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",Напомним,напомнить,"VERB,perf,tran plur,1per,futr,indc",VERB
2,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",23-го,23-го,"NOUN,inan,neut,Fixd sing,nomn",NOUN
3,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",сентября,сентябрь,"NOUN,inan,masc sing,gent",NOUN
4,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",вечером,вечером,ADVB,ADVB
5,Раскрыта тайна препарата в организме утонувшей...,"Напомним , 23-го сентября вечером дочь артиста...",дочь,дочь,"NOUN,anim,femn sing,nomn",NOUN
...,...,...,...,...,...,...
42670,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",Уганде,уганда,"NOUN,inan,femn,Sgtm,Geox sing,datv",NOUN
42671,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",молния,молния,"NOUN,inan,femn sing,nomn",NOUN
42672,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",убила,убить,"VERB,perf,tran femn,sing,past,indc",VERB
42673,Футболисты забили до смерти одноклубника из-за...,"Ранее сообщалось , что во время футбольного м...",10,10,"NUMB,intg",None


Чистота, красота, и только мертвые с косами стоят...мертвые индексы, разумеется. Которые уже совершенно не соответствуют настоящим и будут бесполезны при проверке соседства. Создадим-ка новые

In [ ]:
my_final_dataframe['ind'] = [i for i in range(34947)]

In [ ]:
my_final_dataframe[900:920]

,titles,full_texts,words,lemma,form,POS,ind
1077,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,которые,который,"ADJF,Apro,Subx,Anph plur,nomn",ADJF,900
1078,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,в,в,PREP,PREP,901
1079,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,свою,свой,"ADJF,Apro,Anph femn,sing,accs",ADJF,902
1080,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,очередь,очередь,"NOUN,inan,femn sing,accs",NOUN,903
1081,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,тоже,тоже,ADVB,ADVB,904
1082,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,не,не,PRCL,PRCL,905
1083,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,получат,получить,"VERB,perf,tran plur,3per,futr,indc",VERB,906
1084,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,никаких,никакой,"ADJF,Apro plur,gent",ADJF,907
1085,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,средств,средство,"NOUN,inan,neut plur,gent",NOUN,908
1086,"Девочка сбежала от матери, делавшей из нее рабыню",Вся забота о ней будет пока полностью на плеч...,на,на,PREP,PREP,909


Вуаля! Теперь можно с чистой совестью сохранять в базу данных. Ссв, правда. 

In [ ]:
my_final_dataframe.to_csv('clean_textbase_parsed.csv')

## MAIN

Итак, вот и пришло время для воплощения самой функции мэйн, которая будет искать в нашем корпусе нужные нам слова

Для начала, дернем из документации пайморфи список поддерживаемых тегов:

NOUN	имя существительное

ADJF	имя прилагательное (полное)

ADJS	имя прилагательное (краткое)

COMP	компаратив

VERB	глагол (личная форма)

INFN	глагол (инфинитив)

PRTF	причастие (полное)

PRTS	причастие (краткое)

GRND	деепричастие

NUMR	числительное

ADVB	наречие

NPRO	местоимение-существительное

PRED	предикатив

PREP	предлог

CONJ	союз

PRCL	частица

INTJ	междометие

Пригодится. Теперь осталась только одна маленькая декоративная деталь - класс ошибок, который поможет нам тихо-мирно выходить из ячейки, если пользователь косячит:

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

Его основные преимущества это, во-первых, то, что он не убивает кернел - а это довольно раздражающая штука, - а во-вторых, он тихо выходит и не сообщает пользователю, где там что из-за его головотяпства упало.

Итак, функции-обработчики! Им будут поступать запросы, для которых уже определено, чего от нас хотят - это мельчайшие функции на самом низком уровне пищевой цепи, и задачи у них довольно простые и связаны с непосредственным поиском по базе

In [ ]:
allowed_tags = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']

def get_morphology(dataframe, word):
    #inputs dataframe and str, outputs dataframe and list
    #looks for the same word
    word1 = word.strip('"')
    df_word = dataframe.loc[dataframe['words'] == word1]
    sentences = df_word['full_texts'].tolist()
    if len(sentences)==0:
        print('К сожалению, такого слова нет в моем словаре!')
        raise StopExecution
    else:
        return df_word, sentences
    
def get_tagged(dataframe, word, allowed_tags=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']
):
    #inputs dataframe, list and str, outputs dataframe and list
    #looks for specified lemma with specified POS 
    compl_word = word.split('+')
    tag = set(allowed_tags)&set(compl_word)
    if not tag:
        print('К сожалению, я не знаю этот тег!')
        raise StopExecution
    else:
        current_word = morph.parse(compl_word[0])[0]
        current_word_lemma = current_word.normal_form
        current_word_POS = current_word.tag.POS
        df_word_POS = dataframe.loc[(dataframe['lemma'] == current_word_lemma) & (dataframe['POS'] == current_word_POS)]
        sentences = df_word_POS['full_texts'].tolist()
        return df_word_POS, sentences
    
def get_lemma(dataframe, word):
    #inputs dataframe and str, outputs dataframe and list
    #looks for the same lemma
    word_lem_parsed = morph.parse(word)[0]
    word_lem_lemma = word_lem_parsed.normal_form
    df_lemma = dataframe.loc[(dataframe['lemma'] == word_lem_lemma)]
    sentences = df_lemma['full_texts'].tolist()
    if len(sentences)==0:
        print('К сожалению, такого слова нет в моем словаре!')
        raise StopExecution
    else:
        return df_lemma, sentences
    
def get_pos(dataframe, word):
    #inputs dataframe and str, outputs dataframe and list
    #looks for the same POS
    df_pos = dataframe.loc[(dataframe['POS'] == word)]
    sentences = df_pos['full_texts'].tolist()
    if len(sentences)==0:
        print('К сожалению, слов этой части речи нет в моем словаре!')
        raise StopExecution
    else:
        return df_pos, sentences
    

И вот, момент настал. Две самые крупные рыбы в этом море. Функция работы со словами - та самая станция сортировки, которая определяет, какому именно поисковику перенаправить запрос, и заодно отвечает пользователю, что именно он попросил. Функция работы с запросами токенизирует их по пробелам (зачем множить токенизаторы без необходимости?), и в зависимости от количества токенов выдает ответ: предложения, в которых встретились слова, и заголовки статей, откуда эти предложения к нам попали. Оптимизируем поиск мы постепенным сужением круга поиска - результат поиска по первому слову сужает круг, второе слово ищется уже в выдаче предложений для первого, третье - в выдаче для второго. 

In [ ]:
def work_with_words(dataframe, word, allowed_tags=allowed_tags):
    trick = word[0]
    if trick=='"':
        print('вы запрашиваете слова этой леммы в этой форме')
        return get_morphology(dataframe, word)
    elif '+' in word:
        print('вы запрашиваете лемму с конкретной частью речи')
        return get_tagged(dataframe, word, allowed_tags=allowed_tags)
    elif word in allowed_tags:
        print('вы запрашиваете все слова по тегу этой части речи')
        return get_pos(dataframe, word)
    else:
        print('вы запрашиваете поиск по лемме')
        return get_lemma(dataframe, word)
    
def work_with_quiery(quiery):
    quiery_parsed = quiery.split(' ')
    print(quiery_parsed)
    if len(quiery_parsed)==1:
        print('1')
        answer,_ = work_with_words(my_final_dataframe, quiery_parsed[0], allowed_tags=allowed_tags)
        list_words = []
        for index, row in answer.iterrows():
            answer_word = str('Это слово встретилось в предложении:'+ row['full_texts'] + '. | которое было в статье: ' + row['titles'])
            list_words.append(answer_word)
        return list_words
    
    elif len(quiery_parsed)==2:
        print('2')
        list_sentences = []
        first_answer, first_sentences = work_with_words(my_final_dataframe, quiery_parsed[0], allowed_tags=allowed_tags)
        indexes = first_answer['ind'].tolist()
        indexes = [x + 1 for x in indexes]
        second_answer_pool = my_final_dataframe.loc[my_final_dataframe['full_texts'].isin(first_sentences)]
        second_answer, second_sentences = work_with_words(second_answer_pool, quiery_parsed[1], allowed_tags=allowed_tags)
        second_indexes = second_answer['ind'].tolist()
        right_index = set(indexes)&set(second_indexes)
        if not right_index:
            print('К сожалению, в моем корпусе нет ничего по вашему запросу')
        else:
            for elem in right_index:
                    locate_row = my_final_dataframe.loc[my_final_dataframe['ind'] == elem]
                    for index, row in locate_row.iterrows():
                        answer_word = str('Эти слова встретилось в предложении:'+ row['full_texts'] + '. | которое было в статье: ' + row['titles'])
                        list_sentences.append(answer_word)
        return list_sentences
    
    elif len(quiery_parsed)==3:
        print('3')
        list_sentences = []
        first_answer, first_sentences = work_with_words(my_final_dataframe, quiery_parsed[0], allowed_tags=allowed_tags)
        indexes = first_answer['ind'].tolist()
        indexes = [x + 1 for x in indexes]
        second_answer_pool = my_final_dataframe.loc[my_final_dataframe['full_texts'].isin(first_sentences)]
        second_answer, second_sentences = work_with_words(second_answer_pool, quiery_parsed[1], allowed_tags=allowed_tags)
        second_indexes = second_answer['ind'].tolist()
        right_index = set(indexes)&set(second_indexes)
        if not right_index:
            print('К сожалению, в моем корпусе нет ничего по вашему запросу')
        else:
            right_index = [x + 1 for x in right_index]
            third_answer_pool = my_final_dataframe.loc[my_final_dataframe['full_texts'].isin(second_sentences)]
            third_answer, third_sentences = work_with_words(third_answer_pool, quiery_parsed[2], allowed_tags=allowed_tags)
            third_indexes = third_answer['ind'].tolist()
            master_index = set(right_index)&set(third_indexes)
            if not master_index:
                print('К сожалению, в моем корпусе нет ничего по вашему запросу')
            else:
                for elem in master_index:
                    locate_row = my_final_dataframe.loc[my_final_dataframe['ind'] == elem]
                    for index, row in locate_row.iterrows():
                        answer_word = str('Эти слова встретились в предложении:'+ row['full_texts'] + ', которое было в статье: ' + row['titles'])
                        list_sentences.append(answer_word)
        return list_sentences        
    else:
        print ('К сожалению, я не обрабатываю запросы длиннее трех токенов')
        raise StopExecution

Тадаам! Вот и мэйн. У него здесь дел немного - еще раз проверить, не возникла ли нигде ошибка с ненайденными ответами, которую мы не перехватили, а пользователю видеть и не надо, и предложить пользователю дать запрос. 

In [ ]:
def main():
    quiery = input('Что ищем? Не более трех слов в запросе! ')
    try:
        answer = work_with_quiery(quiery)
        for i in answer:
            print(i)
            print('\n')
    except TypeError:
        print('Что-то пошло не так. Попробуйте переформулировать запрос')
        raise StopExecution

In [ ]:
main()

Что ищем? Не более трех слов в запросе! на содержание девушка
['на', 'содержание', 'девушка']
3
вы запрашиваете поиск по лемме
вы запрашиваете поиск по лемме
вы запрашиваете поиск по лемме
Эти слова встретились в предложении: Вся забота о ней будет пока полностью на плечах опекунов , которые в свою очередь тоже не получат никаких средств на содержание девушки , которое было в статье: Девочка сбежала от матери, делавшей из нее рабыню


